# Construir un modelo de regresión: preparar y visualizar datos

## **Regresión Lineal para Calabazas - Lección 2**
#### Introducción

Ahora que tienes las herramientas necesarias para comenzar a construir modelos de aprendizaje automático con Tidymodels y el Tidyverse, estás listo para empezar a formular preguntas sobre tus datos. Al trabajar con datos y aplicar soluciones de aprendizaje automático, es muy importante saber cómo hacer las preguntas correctas para desbloquear adecuadamente el potencial de tu conjunto de datos.

En esta lección, aprenderás:

-   Cómo preparar tus datos para la construcción de modelos.

-   Cómo usar `ggplot2` para la visualización de datos.

La pregunta que necesitas responder determinará qué tipo de algoritmos de aprendizaje automático utilizarás. Y la calidad de la respuesta que obtengas dependerá en gran medida de la naturaleza de tus datos.

Veamos esto trabajando en un ejercicio práctico.


<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Obra de @allison_horst</figcaption>


<!--![Obra de \@allison_horst](../../../../../../translated_images/unruly_data.0eedc7ced92d2d919cf5ea197bfe0fe9a30780c4bf7cdcf14ff4e9dc5a4c7267.es.jpg)<br>Obra de \@allison_horst-->


## 1. Importar datos de calabazas y convocar el Tidyverse

Necesitaremos los siguientes paquetes para desglosar y analizar esta lección:

-   `tidyverse`: El [tidyverse](https://www.tidyverse.org/) es una [colección de paquetes de R](https://www.tidyverse.org/packages) diseñada para hacer la ciencia de datos más rápida, sencilla y divertida.

Puedes instalarlos con el siguiente comando:

`install.packages(c("tidyverse"))`

El script a continuación verifica si tienes los paquetes necesarios para completar este módulo y los instala por ti en caso de que falte alguno.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

¡Ahora, vamos a activar algunos paquetes y cargar los [datos](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) proporcionados para esta lección!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Un vistazo rápido con `glimpse()` muestra inmediatamente que hay espacios en blanco y una mezcla de cadenas de texto (`chr`) y datos numéricos (`dbl`). La columna `Date` es de tipo carácter y también hay una columna extraña llamada `Package`, donde los datos son una mezcla entre `sacks`, `bins` y otros valores. En realidad, los datos están un poco desordenados 😤.

De hecho, no es muy común recibir un conjunto de datos que esté completamente listo para usar y crear un modelo de ML directamente. Pero no te preocupes, en esta lección aprenderás cómo preparar un conjunto de datos en bruto utilizando bibliotecas estándar de R 🧑‍🔧. También aprenderás varias técnicas para visualizar los datos. 📈📊
<br>

> Un repaso: El operador pipe (`%>%`) realiza operaciones en secuencia lógica pasando un objeto hacia adelante dentro de una función o expresión. Puedes pensar en el operador pipe como si dijera "y luego" en tu código.


## 2. Verificar datos faltantes

Uno de los problemas más comunes que los científicos de datos deben enfrentar es la falta o ausencia de datos. R representa los valores faltantes o desconocidos con un valor especial llamado `NA` (No Disponible).

Entonces, ¿cómo podríamos saber si el marco de datos contiene valores faltantes?  
<br>
-   Una forma sencilla sería usar la función base de R `anyNA`, que devuelve los valores lógicos `TRUE` o `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

¡Genial, parece que falta algo de información! Ese es un buen punto de partida.

-   Otra forma sería usar la función `is.na()` que indica qué elementos individuales de la columna están ausentes con un valor lógico `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Bien, trabajo hecho, pero con un marco de datos tan grande como este, sería ineficiente y prácticamente imposible revisar todas las filas y columnas individualmente😴.

-   Una forma más intuitiva sería calcular la suma de los valores faltantes para cada columna:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Mucho mejor. Faltan algunos datos, pero tal vez no importe para la tarea en cuestión. Veamos qué más nos revela el análisis.

> Además de los increíbles conjuntos de paquetes y funciones, R tiene una documentación muy buena. Por ejemplo, usa `help(colSums)` o `?colSums` para obtener más información sobre la función.


## 3. Dplyr: Una gramática para la manipulación de datos

<p>
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Ilustración de @allison_horst</figcaption>


<!--¡Ilustración de \@allison_horst](../../images/dplyr_wrangling.png)<br/>Ilustración de \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), un paquete del Tidyverse, es una gramática para la manipulación de datos que ofrece un conjunto consistente de verbos que te ayudan a resolver los desafíos más comunes en la manipulación de datos. En esta sección, ¡exploraremos algunos de los verbos de dplyr!  
<br>


#### dplyr::select()

`select()` es una función del paquete `dplyr` que te ayuda a elegir columnas para mantener o excluir.

Para que tu marco de datos sea más fácil de manejar, elimina varias de sus columnas utilizando `select()`, manteniendo solo las columnas que necesitas.

Por ejemplo, en este ejercicio, nuestro análisis incluirá las columnas `Package`, `Low Price`, `High Price` y `Date`. Vamos a seleccionar estas columnas.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` es una función del paquete `dplyr` que te ayuda a crear o modificar columnas, mientras mantienes las columnas existentes.

La estructura general de `mutate` es:

`data %>%   mutate(new_column_name = what_it_contains)`

Vamos a probar `mutate` utilizando la columna `Date` realizando las siguientes operaciones:

1. Convertir las fechas (actualmente de tipo carácter) a un formato de mes (estas son fechas de EE.UU., por lo que el formato es `MM/DD/YYYY`).

2. Extraer el mes de las fechas a una nueva columna.

En R, el paquete [lubridate](https://lubridate.tidyverse.org/) facilita el trabajo con datos de tipo fecha y hora. Entonces, vamos a usar `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` y ver cómo lograr los objetivos mencionados. Podemos eliminar la columna `Date` ya que no la necesitaremos nuevamente en operaciones posteriores.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

¡Genial! 🤩

A continuación, vamos a crear una nueva columna `Price`, que representará el precio promedio de una calabaza. Ahora, tomemos el promedio de las columnas `Low Price` y `High Price` para llenar la nueva columna Price.  
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

¡Siií! 💪

"¡Pero espera!", dirás después de echar un vistazo rápido a todo el conjunto de datos con `View(pumpkins)`, "¡Aquí hay algo raro!" 🤔

Si miras la columna `Package`, las calabazas se venden en muchas configuraciones diferentes. Algunas se venden en medidas de `1 1/9 bushel`, otras en medidas de `1/2 bushel`, algunas por calabaza, otras por libra, y algunas en cajas grandes con anchos variables.

Verifiquemos esto:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

¡Increíble!👏

Las calabazas parecen ser muy difíciles de pesar de manera consistente, así que vamos a filtrarlas seleccionando solo las calabazas que tengan la cadena *bushel* en la columna `Package` y colocarlas en un nuevo marco de datos `new_pumpkins`.


#### dplyr::filter() y stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): crea un subconjunto de los datos que solo contiene las **filas** que cumplen con tus condiciones, en este caso, calabazas con la cadena *bushel* en la columna `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): detecta la presencia o ausencia de un patrón en una cadena de texto.

El paquete [`stringr`](https://github.com/tidyverse/stringr) proporciona funciones simples para operaciones comunes con cadenas de texto.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Puedes ver que hemos reducido a unas 415 filas de datos que contienen calabazas por fanega. 🤩  
<br>


#### dplyr::case_when()

**Pero espera, ¡hay algo más por hacer!**

¿Notaste que la cantidad de bushels varía por fila? Necesitas normalizar los precios para mostrar el precio por bushel, no por 1 1/9 o 1/2 bushel. Es hora de hacer algunos cálculos para estandarizarlo.

Usaremos la función [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) para *mutar* la columna de Precio dependiendo de algunas condiciones. `case_when` te permite vectorizar múltiples declaraciones `if_else()`.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Ahora podemos analizar el precio por unidad basado en su medida en fanegas. Todo este estudio de fanegas de calabazas, sin embargo, demuestra lo `importante` que es `entender la naturaleza de tus datos`.

> ✅ Según [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308), el peso de una fanega depende del tipo de producto, ya que es una medida de volumen. "Una fanega de tomates, por ejemplo, se supone que pesa 56 libras... Las hojas y verduras ocupan más espacio con menos peso, por lo que una fanega de espinacas pesa solo 20 libras." ¡Es todo bastante complicado! No nos molestemos en hacer una conversión de fanegas a libras, y en su lugar fijemos el precio por fanega. Todo este estudio de fanegas de calabazas, sin embargo, demuestra lo importante que es entender la naturaleza de tus datos.
>
> ✅ ¿Notaste que las calabazas vendidas por media fanega son muy caras? ¿Puedes averiguar por qué? Pista: las calabazas pequeñas son mucho más caras que las grandes, probablemente porque hay muchas más de ellas por fanega, dado el espacio no utilizado que ocupa una gran calabaza hueca para pastel.


Ahora, por último, solo por el simple gusto de la aventura 💁‍♀️, movamos también la columna de Mes a la primera posición, es decir, `antes` de la columna `Paquete`.

Se utiliza `dplyr::relocate()` para cambiar las posiciones de las columnas.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

¡Buen trabajo!👌 Ahora tienes un conjunto de datos limpio y ordenado sobre el cual puedes construir tu nuevo modelo de regresión.  
<br>


## 4. Visualización de datos con ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infografía por Dasani Madipalli</figcaption>


<!--![Infografía por Dasani Madipalli](../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.es.png){width="600"}-->

Hay un dicho *sabio* que dice así:

> "El gráfico simple ha aportado más información a la mente del analista de datos que cualquier otro dispositivo." --- John Tukey

Parte del rol del científico de datos es demostrar la calidad y naturaleza de los datos con los que está trabajando. Para ello, a menudo crean visualizaciones interesantes, o gráficos, diagramas y tablas, que muestran diferentes aspectos de los datos. De esta manera, pueden mostrar visualmente relaciones y brechas que de otro modo serían difíciles de descubrir.

Las visualizaciones también pueden ayudar a determinar la técnica de aprendizaje automático más adecuada para los datos. Un diagrama de dispersión que parece seguir una línea, por ejemplo, indica que los datos son un buen candidato para un ejercicio de regresión lineal.

R ofrece varios sistemas para crear gráficos, pero [`ggplot2`](https://ggplot2.tidyverse.org/index.html) es uno de los más elegantes y versátiles. `ggplot2` te permite componer gráficos **combinando componentes independientes**.

Comencemos con un simple diagrama de dispersión para las columnas Price y Month.

En este caso, comenzaremos con [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), proporcionaremos un conjunto de datos y un mapeo estético (con [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)) y luego añadiremos capas (como [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) para los diagramas de dispersión.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

¿Es este un gráfico útil 🤷? ¿Hay algo en él que te sorprenda?

No es particularmente útil, ya que todo lo que hace es mostrar tus datos como una dispersión de puntos en un mes determinado.  
<br>


### **¿Cómo lo hacemos útil?**

Para que los gráficos muestren datos útiles, generalmente necesitas agrupar los datos de alguna manera. Por ejemplo, en nuestro caso, encontrar el precio promedio de las calabazas para cada mes proporcionaría más información sobre los patrones subyacentes en nuestros datos. Esto nos lleva a otro vistazo rápido de **dplyr**:

#### `dplyr::group_by() %>% summarize()`

La agregación agrupada en R se puede calcular fácilmente usando

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` cambia la unidad de análisis del conjunto de datos completo a grupos individuales, como por mes.

-   `dplyr::summarize()` crea un nuevo marco de datos con una columna para cada variable de agrupación y una columna para cada estadística resumida que hayas especificado.

Por ejemplo, podemos usar `dplyr::group_by() %>% summarize()` para agrupar las calabazas en grupos basados en la columna **Month** y luego encontrar el **precio promedio** para cada mes.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

¡Sucinto!✨

Las características categóricas, como los meses, se representan mejor utilizando un gráfico de barras 📊. Las capas responsables de los gráficos de barras son `geom_bar()` y `geom_col()`. Consulta `?geom_bar` para obtener más información.

¡Vamos a crear uno!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩¡Esta es una visualización de datos más útil! Parece indicar que el precio más alto de las calabazas ocurre en septiembre y octubre. ¿Cumple eso con tus expectativas? ¿Por qué sí o por qué no?

¡Felicidades por terminar la segunda lección 👏! ¡Preparaste tus datos para la construcción del modelo y luego descubriste más información utilizando visualizaciones!



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por lograr precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
